# Import Library

In [ ]:
import matplotlib
%matplotlib inline

import pandas as pd
import numpy as np
import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt
import os
import sys
import importlib

import SimpleITK as sitk
from joblib import Parallel, delayed


import sys
sys.path.append('/lustre/grp/wxqlab/zhouw/JupyterCode')
import cellcloud3d as cc

import scanpy as sc
sc.settings.verbosity = 3
sc.logging.print_header()
sc.settings.set_figure_params(dpi=100, dpi_save=200, fontsize=10, facecolor='white') 
#sc.settings.figdir='./'

# SITK

In [ ]:
importlib.reload(cc.rg)
importlib.reload(cc)
importlib.reload(cc.rg)
importlib.reload(cc.rg._sitkreg)
importlib.reload(cc.rg._features)
importlib.reload(cc.tf)
importlib.reload(cc.tf._transi)
importlib.reload(cc.tf._transp)

In [ ]:
import skimage as ski
from skimage import transform
from skimage import data
from skimage import transform
from skimage import img_as_float

tform = transform.EuclideanTransform(
    rotation=np.pi / 12.,
    translation = (100, -20)
    )
print(tform.params)
print("*"*30)
# tform = transform.SimilarityTransform(
#     scale=0.5,
#     rotation=np.pi/12,
#     translation=(100, 50))
# print(tform.params)

img = img_as_float(data.chelsea())
tf_img = transform.warp(img, tform.inverse)
ifix = img #ski.color.rgb2gray(img)
imov = tf_img #ski.color.rgb2gray(tf_img)
ifix = ski.color.rgb2gray(img)
imov = ski.color.rgb2gray(tf_img)

fixed_image = cc.rg.sitkregist.image2sitk(ifix,2)
moving_image = cc.rg.sitkregist.image2sitk(imov,2)

rgsitk = cc.rg.sitkregist(transtype='rigid', dimension=2)
mov_out, tparas = rgsitk.regist(fixed_image, moving_image, 
                             msp=0.8, 
                            #  centralRegionRadius=20,
                             optimizer_type = 'RSGD', #'Exhaustive',
                             optimizer_kargs={ 
                                    'numberOfSteps': [5,5,5,5],
                                    'stepLength' : 1,
                                    'learningRate':3,
                                    'gradientMagnitudeTolerance':1e-7,
                                    'minStep': 1e-6,
                                },
                             matrix_type='CORR')
regmeth = rgsitk.reg_method

moving_resampled = sitk.Resample(
    moving_image,
    fixed_image,
    tparas,
    sitk.sitkLinear,
    0.0,
    moving_image.GetPixelID())


cc.pl.qview(ifix, imov, sitk.GetArrayFromImage(mov_out), sitk.GetArrayFromImage(moving_resampled), ncols=4)
print(tparas)
print('*'*30)
print(regmeth)